<a href="https://colab.research.google.com/github/DmitriMAI/to_future/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22GPT_introduction_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [19]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [14]:
from datasets import load_dataset, Dataset

In [15]:
dataset = load_dataset("text", data_files={"train": "./train_dataset.txt"}, sample_by="document")

Using custom data configuration default-974fcbb212fe5ba9
Found cached dataset text (C:/Users/Dima/.cache/huggingface/datasets/text/default-974fcbb212fe5ba9/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
joke_datasets = []

In [19]:
k = 0
for index, item in enumerate(dataset['train'][0]['text'].split('\n\n\n\n')):
    k = k +1
    if (k > 10): #ВОт тут длинна датасета !!!!!
        break
    joke_datasets.append({"text":item})

In [20]:
dataset = Dataset.from_list(joke_datasets, split="train+test")

In [21]:
dataset[0]

{'text': '- Как водичка ?\n- А я здесь как женшина сижу, а не как термометр.'}

In [22]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"]))

  0%|          | 0/11 [00:00<?, ?ex/s]

In [23]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [24]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuner",
    learning_rate=2e-3,
    per_device_train_batch_size=2, # если позволяет память - можно сделать больше
    per_device_eval_batch_size=2,
    num_train_epochs=10, # сколько будет пройден датасет полностью
    weight_decay=0.01,
)

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
        optimizers = (torch.optim.AdamW(model.parameters(),lr=2e-3),None) # Optimizer and lr scheduler
)

In [26]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Assigning [PAD] to the pad_token key of the tokenizer


0

In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 11
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 60


  0%|          | 0/60 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 56.0245, 'train_samples_per_second': 1.963, 'train_steps_per_second': 1.071, 'train_loss': 4.937605285644532, 'epoch': 10.0}


TrainOutput(global_step=60, training_loss=4.937605285644532, metrics={'train_runtime': 56.0245, 'train_samples_per_second': 1.963, 'train_steps_per_second': 1.071, 'train_loss': 4.937605285644532, 'epoch': 10.0})

In [1]:
from transformers import AutoConfig
label2id= {
    "negative":0,
    "positive":1,
}
id2label= {
    0:"negative",
    1:"positive",
}
config = AutoConfig.from_pretrained("./checkpoint-125000", label2id=label2id, id2label=id2label)

In [2]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("./checkpoint-125000")
model = GPT2LMHeadModel.from_pretrained("./checkpoint-125000",config=config).to(DEVICE) # Загружаем обученную модель из чекпоинтов

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:

text = "Денис не любит аниме"
input_ids = tokenizer.encode(text, return_tensors="pt").to("cuda:0")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100, pad_token_id=tokenizer.eos_token_id,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Денис не любит аниме.
- Ты с ума сошла!
- По мне хоть Камасутра, где все люди - мадонны.
- Да, но она такая добрая, такая милая...
- Да, но она страшная.
- Знаешь, я бы на твоем месте не стала так убиваться.
- Но я бы не стала так жалеть.
- Вот именно: Камасутра - добрая, но страшная.
- Да, но она страшная.



In [21]:

# Пример генерации с помощью beam-search
out = model.generate(input_ids, do_sample=False, num_beams=5, max_length=100)

# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Денис не любит аниме.
- По какой-то странной причине.
- Например?
- Он считает, что все женщины - ужасные уродины.
- То есть?
- В том и весь фокус. Он считает, что все хорошие девушки - дуры.
- То есть?
- В том и весь фокус. Он считает, что все хорошие девушки - дуры.
- То есть?
- В том и весь фокус. Он считает, что все хорошие девушки
